In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.simple.load_model import *
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight
from src.utils.save_model import load_model

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [2]:
# color, affine + color, affine
#model_ids = ["c55ed5c6-e879-11ea-9a58-0242ac110005", "2eb47be0-e88c-11ea-9a58-0242ac110005", "74d8220a-e6c2-11ea-b084-0242ac110005"]

experiment_ids = ["f5345e7e-e603-11ea-b461-0242ac110005",
             "74d8220a-e6c2-11ea-b084-0242ac110005",
             #"7f0b1972-e5e5-11ea-a85b-0242ac110005",
             #"ba5364e6-e949-11ea-87eb-0242ac110005",
             "bd5c2e9a-e9ec-11ea-ae3a-0242ac110005"
]

experiment_ids = [
    "bd5c2e9a-e9ec-11ea-ae3a-0242ac110005",
    "ba5364e6-e949-11ea-87eb-0242ac110005",
    "74d8220a-e6c2-11ea-b084-0242ac110005",
    "e506d8f0-ea0f-11ea-8dbb-0242ac110005",
    "f5345e7e-e603-11ea-b461-0242ac110005",
    "5dcb7ecc-e4e2-11ea-96c6-0242ac110005",
    "2eb47be0-e88c-11ea-9a58-0242ac110005",
    "7f0b1972-e5e5-11ea-a85b-0242ac110005",
]

experiment_names = []


In [3]:
experiments = {} 
experiment_dicts = {} 
for experiment_id in experiment_ids:
    exp_dict = get_experiment_from_logs(experiment_id=experiment_id)
    experiment = rebuild_experiment(exp_dict)
    #exp_dict["test_again"] = reevaluate(experiment)
    #exp_dict["test_difference"] = sum(difference_test_results(exp_dict["test"], exp_dict["test_again"]).values())
    #print("test_difference is ", exp_dict["test_difference"])
    #experiment_dicts[exp_dict["name"]] = exp_dict
    experiments[exp_dict["name"]] =  experiment
        
experiments.keys()

Using weights file DenseNet121_2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200829-114238.h5 to load model...
Using weights file DenseNet121_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200828-161545.h5 to load model...
Using weights file DenseNet121_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200825-110224.h5 to load model...
Using weights file DenseNet169_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200829-155417.h5 to load model...
Using weights file DenseNet121_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled_20200824-121846.h5 to load model...
Using weights file DenseNet121_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled_20200823-014547.h5 to load model...
Using weights file DenseNet121_Che

{'DenseNet121_2_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled': <src.architectures.benchmarks.benchmark.Experiment at 0x7fdba47363c8>,
 'DenseNet121_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled': <src.architectures.benchmarks.benchmark.Experiment at 0x7fdb95c39b38>,
 'DenseNet121_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled': <src.architectures.benchmarks.benchmark.Experiment at 0x7fdb891c4da0>,
 'DenseNet169_Chexpert_CWBCE_L1Normed_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled': <src.architectures.benchmarks.benchmark.Experiment at 0x7fdb8215ec50>,
 'DenseNet121_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_5LR1_LF1_SGD_Upsampled': <src.architectures.benchmarks.benchmark.Experiment at 0x7fd8b5e57630>,
 'DenseNet121_Chexpert_CWBCE_L1Normed_E3_B32_C0_N12_AugAffine_U75_D256_DS9505_5LR1_LF1_S

In [4]:
predictions = {}
for exp_name, exp in experiments.items():
    testgen = exp.benchmark.testgen
    testgen.on_epoch_end()

    predictions[exp_name] = exp.model.predict(
        testgen, steps=len(testgen), verbose=1)

234/234 [==============================] - 5s 20ms/step


In [19]:
list(predictions.values())[0]

array([[0.14204814, 0.05922524, 0.04125153, ..., 0.00329158, 0.024773  ,
        0.20067492],
       [0.12992185, 0.04159359, 0.00746781, ..., 0.01144745, 0.01837277,
        0.05012335],
       [0.08564375, 0.03324569, 0.02694898, ..., 0.01777107, 0.03584199,
        0.09752537],
       ...,
       [0.4016886 , 0.28435183, 0.07799099, ..., 0.00209289, 0.01425389,
        0.1449461 ],
       [0.26744035, 0.16901775, 0.03899708, ..., 0.00441077, 0.07441124,
        0.04030176],
       [0.9253501 , 0.0333859 , 0.12178481, ..., 0.15148972, 0.02372288,
        0.03865588]], dtype=float32)

In [22]:
y_pred = tf.math.reduce_mean(np.array(list(predictions.values())), axis=0)
y_pred.shape

TensorShape([234, 12])

In [20]:
y_pred=list(predictions.values())[0]

In [23]:
metrics = {}
groundtruth_label = exp.benchmark.testgen.get_labels_nonan()
for metric in exp.benchmark.metrics:
    metric.reset_states()
    metric.update_state(tensorflow.constant(groundtruth_label), tensorflow.constant(y_pred))
    metrics[metric.name] = metric.result().numpy()
metrics

{'auc': 0.76924914,
 'precision': 0.6911765,
 'recall': 0.08608059,
 'f2_score': 0.104351684,
 'binary_accuracy': 0.8148149,
 'accuracy_enlarged_cardiomediastinum': 0.53846157,
 'accuracy_cardiomegaly': 0.7393162,
 'accuracy_lung_opacity': 0.46153846,
 'accuracy_lung_lesion': 0.98717946,
 'accuracy_edema': 0.86752135,
 'accuracy_consolidation': 0.8632479,
 'accuracy_pneumonia': 0.96153843,
 'accuracy_atelectasis': 0.66239315,
 'accuracy_pneumothorax': 0.96581197,
 'accuracy_pleural_effusion': 0.7777778,
 'accuracy_pleural_other': 0.974359,
 'accuracy_fracture': 0.97863245,
 'auc_enlarged_cardiomediastinum': 0.48943114,
 'auc_cardiomegaly': 0.7946492,
 'auc_lung_opacity': 0.92353755,
 'auc_lung_lesion': 0.85622317,
 'auc_edema': 0.9340976,
 'auc_consolidation': 0.9348711,
 'auc_pneumonia': 0.8108407,
 'auc_atelectasis': 0.8461851,
 'auc_pneumothorax': 0.7970133,
 'auc_pleural_effusion': 0.925686,
 'auc_pleural_other': 0.9184549,
 'auc_fracture': 0.0,
 'precision_enlarged_cardiomediastin

In [17]:
for preds in predictions.values():
    print(auc_metric(groundtruth_label, preds))

0.80215734
0.74252623
0.7985672
0.8036845
0.79839516
0.80233026
0.8115233
0.80394506


In [16]:
def auc_metric(y_true, y_pred):
    m = tf.keras.metrics.AUC()
    m.update_state(y_true, y_pred)
    return m.result().numpy()